In [1]:
import pandas as pd

# 데이터 경로

data_path = './porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv' , index_col = 'id')
test = pd.read_csv(data_path + 'test.csv' , index_col = 'id')
submission = pd.read_csv(data_path + 'sample_submission.csv' , index_col ='id')


In [2]:
all_data = pd.concat([train , test] , ignore_index= True)
all_data = all_data.drop('target' , axis = 1) # 타깃값 제거

all_data

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,2,2,5,1,0,0,1,0,0,0,...,9,1,5,8,0,1,1,0,0,1
1,1,1,7,0,0,0,0,1,0,0,...,3,1,1,9,0,1,1,0,1,0
2,5,4,9,1,0,0,0,1,0,0,...,4,2,7,7,0,1,1,0,1,0
3,0,1,2,0,0,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,0,2,0,1,0,1,0,0,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488023,0,1,6,0,0,0,1,0,0,0,...,4,2,3,4,0,1,0,0,1,0
1488024,5,3,5,1,0,0,0,1,0,0,...,6,2,2,11,0,0,1,1,0,0
1488025,0,1,5,0,0,1,0,0,0,0,...,5,2,2,11,0,1,1,0,0,0
1488026,6,1,5,1,0,0,0,0,1,0,...,1,1,2,7,1,1,0,0,0,0


In [3]:
all_features = all_data.columns # 전체 피처
all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

## 명목형 피처 원-핫 인코딩

In [4]:
from sklearn.preprocessing import OneHotEncoder

# 명목형 피처 추출
cat_features = [feature for feature in all_features if 'cat' in feature]
# 이름에 cat이 포함된 피처가 명목형 피처이다.
onehot_encoder = OneHotEncoder() # 원-핫 인코더 객체 생성

# 인코딩
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

encoded_cat_matrix

<1488028x184 sparse matrix of type '<class 'numpy.float64'>'
	with 20832392 stored elements in Compressed Sparse Row format>

In [5]:
# 추가로 제거할 피처
drop_features = ['ps_ind14' , 'ps_ind_10_bin' , 'ps_ind_11_bin' , 'ps_ind_12_bin' , 'ps_ind_13_bin' , 'ps_car_14' ]

# 1> 명목형 피처 , 2> calc 분류의 피처 , 3> 추가 제거할 피처를 제외한 피처

remaining_features = [feature for feature in all_features
                      if ('cat' not in feature and
                          'calc' not in feature and
                          feature not in drop_features)]
# cat(명목형) 피처 , calc(피처) , 추가 제거할 6개피처를 제외한 나머지 피처를 remaining_features 에 저장

In [6]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]) , encoded_cat_matrix] , format= 'csr')

all_data_sprs

# CSR  형식으로 바꾸어 hstack()으로 행렬을 수평 방향으로 합친다.

<1488028x202 sparse matrix of type '<class 'numpy.float64'>'
	with 37644877 stored elements in Compressed Sparse Row format>

In [7]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기

X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train :]

y = train['target'].values

## 지니계수

In [8]:
# 지니계수란?

# 소득 불평등 정도를 나타내는 지표. 지니계수가 작을수록 소득 수준이 평등하고, 클수록 불평등함을 의미한다.

In [9]:
# 정규화 지니계수 계산 함수

# 정규화란 값의 범위를 0~1 사이로 조정한다는 의미, 정규화 지니계수는 값이 0에 가까울수록 성능이 나쁘고, 1에 가까울 수록 성능이 좋다는 의미가 된다.

In [10]:
import numpy as np

def eval_gini(y_true , y_pred):
    # 실제값과 예측값의 크기가 서로 같은지 확인(값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0] # 데이터 개수
    L_mid = np.linspace(1/ n_samples ,1 , n_samples) # 대각선 값

    # 1) 예측값에 대한 지니계수

    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선

    G_pred = np.sum(L_mid - L_pred) # 예측값에 대한 지니계수

    # 2) 예측이 완벽할 때 지니계쑤

    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true) # 예측이 완벽할 때 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [11]:
# LightGBM 용 gini() 함수

def gini(preds , dtrain):
    labels = dtrain.get_label()

    return 'gini' , eval_gini(labels , preds ) , True

# 'gini' : 평가지표이름 , eval_gini(labels,preds) : 평가점수 , True : 평가 점수가 높을수록 좋은지 여부

## 모델 훈련 및 성능 검증

In [12]:
# OOF(Out of Fold prediction) 예측 방식

# K 폴드 교차 검증을 수행하면서 각 폴드마다 테스트 데이터로 예측하는 방식이다.

# K 폴드 교차 검ㅈ응을 하면서 폴드마다 1> 훈련 데이터로 모델을 훈련하고, 2> 검증 데이터로 모델 성능을 측정하며 , 3> 테스트 데이터로 최종 타깃 확률도 예측한다. 훈련된 모델로 마지막에 한 번만 예측하는 것이 아니다. 각 폴드별 모델로 여러번 예측해 평균을 내는 방식이다.

In [13]:
# OOF 방식으로 LightGBM 훈련
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기

folds = StratifiedKFold(n_splits= 5 , shuffle= True , random_state= 1991)

# 층화 K 폴드 교차 검증기는 타깃값이 불균형하므로 K폴드가 아닌 층화 K폴드를 수행하는 게 바람직하다. 층화 K폴드는 타깃값이 균등하게
# 폴드를 나누는 방식이기 때문이다.


# n_splits 파라미터로 전달한 수만큼 폴드를 나눈다. 여기서는 5개로 나누었다. shuffle = True 를 전달하면 폴드를 나눌때 데이터를 섞어준다.

In [14]:
# LightGBM의 하이퍼파라미터를 설정한다. LightGBM은 하이퍼파라미터를 갖고 있지만, 여기서는 4가지만 설정한다.

params = {'objective' : 'binary' , 'learning_rate' : 0.01 , 'force_row_wise' : True , 'random_state' : 0}

# 이진분류 문제이므로 objective 파라미터는 binary로 설정했다. 학습률은 0.01로, 랜덤 스테이트 값은 9으로 설정했다.
# force_row_wise : True 는 경고 문구를 없애려고 추가한 파라미터이다.

In [15]:
# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0])

# ==> oof_val_preds 는 검증 데이터를 활용해 예측한 확률값을 저장하는 배열이다. K 폴드로 나누어도 훈련 데이터 전체가 결국엔 한 번씩 검증 데이터로 활용된다. 따라서 oof_val_preds 배열 크기는 훈련 데이터와 같아야 한다.
# 훈련 데이터 개수는 X.shpae[0]으로 구한다.

# OOF 방식으로 훈련된 모델로 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0])

# oof_test_preds는 테스트 데이터를 활용해 예측한 확률값을 저장하는 배열이다. 최종 제출에 사용할 값이므로 크기는 테스트 데이터와 같아야한다. 테스트 데이터 개수는 X_test.shape[0]으로 구한다.

In [16]:
# !pip install --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org lightgbm

In [17]:
import lightgbm as lgb

# OOF 방식으로 모델 훈련 , 검증 , 예측

for idx, (train_idx , valid_idx) in enumerate(folds.split(X, y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40 , f'폴드 {idx +1} / 폴드 {folds.n_splits}' , '#'*40)

    # 훈련용 데이터, 검증용 데이터 설정
    X_train , y_train = X[train_idx] , y[train_idx] # 훈련용 데이터
    X_valid , y_valid = X[valid_idx] , y[valid_idx] # 검증용 데이터

    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train , y_train) # LightGBM 전용 훈련 데이터 셋
    dvalid = lgb.Dataset(X_valid , y_valid) # LightGBM 전용 검증 데이터 셋

    # LightGBM 모델 훈련
    lgb_model = lgb.train(params = params , # 훈련용 하이퍼파라미터
                          train_set = dtrain, # 훈련 데이터 셋
                          num_boost_round = 1000, # 부스팅 반복 횟수
                          valid_sets=  dvalid ,  # 성능 평가용 검증 데이터 셋
                          feval = gini, # 검증용 평가지표
                          early_stopping_rounds = 100, # 조기종료 조건
                          verbose_eval = 100 ) # 100번째마다 점수 출력

    # 테스트 데이터를 활용해 OOF 예측

    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits

    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측

    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    # 검증 데이터 예측 확률에 대한 정규화 지니계수

    gini_score = eval_gini(y_valid , oof_val_preds[valid_idx])
    print(f'폴드 {idx +1} 지니계수 : {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1100
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 201
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.153356	valid_0's gini: 0.261373
[200]	valid_0's binary_logloss: 0.152424	valid_0's gini: 0.27589
[300]	valid_0's binary_logloss: 0.152031	valid_0's gini: 0.281949
[400]	valid_0's binary_logloss: 0.151802	valid_0's gini: 0.286194
[500]	valid_0's binary_logloss: 0.151737	valid_0's gini: 0.286881
[600]	valid_0's binary_logloss: 0.151686	valid_0's gini: 0.287701
[700]	valid_0's binary_logloss: 0.151677	valid_0's gini: 0.287901
Early stopping, best iteration is:
[655]	valid_0's binary_logloss: 0.151674	valid_0's gini: 0.287982
폴드 1 지니계수 : 0.2879824293420261

#########

In [18]:
print('OOF 검증 데이터 지니계수 : ' , eval_gini(y , oof_val_preds))

OOF 검증 데이터 지니계수 :  0.27992426994281666


In [19]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')

## 성능 개선

In [20]:
all_data = pd.concat([train , test] , ignore_index = True)
all_data = all_data.drop('target' , axis =1 ) # 타깃값 제거

all_features = all_data.columns # 전체 피처

all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

In [21]:
from sklearn.preprocessing import OneHotEncoder

# 명목형 피처

cat_features = [feature for feature in all_features if 'cat' in feature]

# 원-핫 인코딩 적용

onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

## 파생 피처 추가

In [22]:
# 탐색적 데이터 분석과정에서는 필요 없는 피처를 추리는 것 외에 특별한 피처 엔지니어링 건을 찾아내지 못했다.

In [23]:
# 첫번째 , 한 데이터가 가진 결측값 개수를 파생 피처로 만들어본다. -1 이 결측값이므로 결측값 개수를 구하려면 -1 개수를 구하면 된다.

# 데이터 하나당 결측값 개수를 파생 피처로 추가

all_data['num_missing'] = (all_data == -1).sum(axis = 1)

In [24]:
# 명목형 피처 , calc 분류의 피처를 제외한 피처

remaining_features = [feature for feature in all_features if ('cat' not in feature and 'calc' not in feature)]

# num_missing을 remaining_features 에 추가

remaining_features.append('num_missing')

remaining_features

['ps_ind_01',
 'ps_ind_03',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_10_bin',
 'ps_ind_11_bin',
 'ps_ind_12_bin',
 'ps_ind_13_bin',
 'ps_ind_14',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_14',
 'ps_car_15',
 'num_missing']

In [25]:
# 두번째, ind 분류의 피처들을 살펴본다. 모든 ind 피처 값을 연결해서 새로운 피처를 만들려고한다.

# 예를들어 , ps_ind_01 , ps_ind_02_car , ps_ind_03의 값이 각각 2, 3, 5 라면 모든 값을 연결해 2_2_5_로 만든다.

# ind 피처가 총 18개이므로 18개 값이 연결된다.

ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True

for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str) + '-'
        is_first_feature = False
    else:
        all_data['mix_ind'] += all_data[ind_feature].astype(str) + '-'

In [26]:
all_data['mix_ind']

0          2-2-5-1-0-0-1-0-0-0-0-0-0-0-11-0-1-0-
1           1-1-7-0-0-0-0-1-0-0-0-0-0-0-3-0-0-1-
2          5-4-9-1-0-0-0-1-0-0-0-0-0-0-12-1-0-0-
3           0-1-2-0-0-1-0-0-0-0-0-0-0-0-8-1-0-0-
4           0-2-0-1-0-1-0-0-0-0-0-0-0-0-9-1-0-0-
                           ...                  
1488023     0-1-6-0-0-0-1-0-0-0-0-0-0-0-2-0-0-1-
1488024    5-3-5-1-0-0-0-1-0-0-0-0-0-0-11-1-0-0-
1488025     0-1-5-0-0-1-0-0-0-0-0-0-0-0-5-0-0-1-
1488026    6-1-5-1-0-0-0-0-1-0-0-0-0-0-13-1-0-0-
1488027    7-1-4-1-0-0-0-0-1-0-0-0-0-0-12-1-0-0-
Name: mix_ind, Length: 1488028, dtype: object

In [27]:
# 세번째 , 명목형 피처의 고윳값별 개수를 새로운 피처로 추가한다. 고윳값별 개수는 value_counts()로 구한다.

# ps_ind_02_cat 피처의 고윳값별 개수 코드

all_data['ps_ind_02_cat'].value_counts()

 1    1079327
 2     309747
 3      70172
 4      28259
-1        523
Name: ps_ind_02_cat, dtype: int64

In [28]:
# 명목형 피처의 고윳값별 개수를 파생 피처로 생성
all_data['ps_ind_02_cat'].value_counts().to_dict()

{1: 1079327, 2: 309747, 3: 70172, 4: 28259, -1: 523}

In [29]:
cat_count_features = []

for feature in cat_features+['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x: val_counts_dict[x])

    cat_count_features.append(f'{feature}_count')

In [30]:
cat_count_features

['ps_ind_02_cat_count',
 'ps_ind_04_cat_count',
 'ps_ind_05_cat_count',
 'ps_car_01_cat_count',
 'ps_car_02_cat_count',
 'ps_car_03_cat_count',
 'ps_car_04_cat_count',
 'ps_car_05_cat_count',
 'ps_car_06_cat_count',
 'ps_car_07_cat_count',
 'ps_car_08_cat_count',
 'ps_car_09_cat_count',
 'ps_car_10_cat_count',
 'ps_car_11_cat_count',
 'mix_ind_count']

In [31]:
from scipy import sparse

# 필요 없는 피처들
drop_features = ['ps_ind_14' , 'ps_ind_10_bin' , 'ps_ind_11_bin' , 'ps_ind_12_bin' , 'ps_ind_13_bin' , 'ps_car_14']

# remaining_features , cat_count_features 에서 drop_features를 제거한 데이터

all_data_remaining = all_data[remaining_features + cat_count_features].drop(drop_features , axis = 1)

# 데이터 합치기

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining) , encoded_cat_matrix] ,format = 'csr')

## 데이터 나누기

In [32]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기

X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train :]

y = train['target'].values

## 하이퍼파라미터 최적화

In [33]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리(베이지안 최적화 수행용)

X_train , X_valid , y_train , y_valid = train_test_split(X,y, test_size=0.2 , random_state=0)

# 베이지안 최적화용 데이터셋

bayes_dtrain = lgb.Dataset(X_train , y_train)
bayes_dvalid = lgb.Dataset(X_valid, y_valid)

In [34]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
param_bounds = {'num_leaves' : (30 , 40) ,
                'lambda_l1' : (0.7 , 0.9),
                'lambda_l2' : (0.9 , 1),
                'feature_fraction' : (0.6 , 0.7),
                'bagging_fraction' : (0.6 , 0.9),
                'min_child_samples' : (6 , 10) ,
                'min_child_weight' : (10 , 40)}


# 값이 고정된 하이퍼파라미터

fixed_params = {'objective' : 'binary' ,
                'learning_rate' : 0.005,
                'bagging_freq' : 1,
                'force_row_wise' : True,
                'random_state' : 1991}

## 베이지안 최적화용 평가지표 계산 함수 작성

In [35]:
def eval_function(num_leaves , lambda_l1 , lambda_l2 , feature_fraction , bagging_fraction , min_child_samples , min_child_weight) :

# 최적화하려는 평가지표(지니계수) 계산 함수

# 베이지안 최적화를 수행할 하이퍼파라미터

    params = {'num_leaves' : int(round(num_leaves)) ,
              'lambda_l1' : lambda_l1,
              'lambda_l2' : lambda_l2 ,
              'feature_fraction' : feature_fraction ,
              'bagging_fraction' : bagging_fraction,
              'min_child_samples' : int(round(min_child_samples)) ,
              'min_child_weight' : min_child_weight,
              'feature_pre_filter' : False}

    #하이퍼파라미터도 추가
    params.update(fixed_params)

    print('하이퍼파라미터 : ' , params)

    # LightGBM 모델 훈련
    lgb_model = lgb.train(params = params ,
                          train_set = bayes_dtrain,
                          num_boost_round= 2500,
                          valid_sets= bayes_dvalid,
                          feval = gini,
                          early_stopping_rounds= 300,
                          verbose_eval= False)
    # 검증 데이터로 예측 수행
    preds = lgb_model.predict(X_valid)

    # 지니계수 계산
    gini_score = eval_gini(y_valid, preds)
    print(f'지니계수 : {gini_score}\n')

    return gini_score

## 최적화 수행

In [36]:
!pip install --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org colorama==0.4.4


[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
!pip install --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org bayesian-optimization==1.4.0

  Using cached bayesian_optimization-1.4.0-py3-none-any.whl (17 kB)



[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f = eval_function, # 평가지표 계산 함수
                                pbounds = param_bounds, # 하이퍼파라미터 범위
                                random_state = 0 )

In [39]:
# 베이지안 최적화 수행

optimizer.maximize(init_points=  3 , n_iter = 6)

C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
하이퍼파라미터 :  {'num_leaves': 34, 'lambda_l1': 0.8205526752143287, 'lambda_l2': 0.9544883182996897, 'feature_fraction': 0.6715189366372419, 'bagging_fraction': 0.7646440511781974, 'min_child_samples': 8, 'min_child_weight': 29.376823391999682, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1555
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.2855811556220905

| 1         | 0.2856    | 0.7646    |

C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 1555
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.2828993761731121

| 4         | 0.2829    | 0.8978    | 0.6594    | 0.8445    | 0.9234    | 8.619     | 10.55     | 30.09     |
하이퍼파라미터 :  {'num_leaves': 37, 'lambda_l1': 0.7738449330497988, 'lambda_l2': 0.9032695189818599, 'feature_fraction': 0.6606341064409726, 'bagging_fraction': 0.7666713964943057, 'min_child_samples': 9, 'min_child_weight': 29.306172421380474, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786


C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1555
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.2854766974907317

| 7         | 0.2855    | 0.8363    | 0.686     | 0.8434    | 0.948     | 6.002     | 39.77     | 36.8      |
하이퍼파라미터 :  {'num_leaves': 30, 'lambda_l1': 0.7759269600824816, 'lambda_l2': 0.9, 'feature_fraction': 0.7, 'bagging_fraction': 0.6, 'min_child_samples': 10, 'min_child_weight': 28.21283616384366, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1555


C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.2847964155782387

| 8         | 0.2848    | 0.6       | 0.7       | 0.7759    | 0.9       | 10.0      | 28.21     | 30.0      |
하이퍼파라미터 :  {'num_leaves': 36, 'lambda_l1': 0.7, 'lambda_l2': 1.0, 'feature_fraction': 0.7, 'bagging_fraction': 0.9, 'min_child_samples': 6, 'min_child_weight': 33.98699093132045, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1555


C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.283740750776938

| 9         | 0.2837    | 0.9       | 0.7       | 0.7       | 1.0       | 6.0       | 33.99     | 35.84     |


## 결과확인

In [40]:
# 평가함수 점수가 최대일 대 하이퍼파라미터
max_params = optimizer.max['params']
max_params

{'bagging_fraction': 0.6213108174593661,
 'feature_fraction': 0.608712929970154,
 'lambda_l1': 0.7040436794880651,
 'lambda_l2': 0.9832619845547939,
 'min_child_samples': 9.112627003799401,
 'min_child_weight': 36.10036444740457,
 'num_leaves': 39.78618342232764}

In [41]:
# 정수형 하이퍼파라미터 변환

max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))

In [42]:
# 값이 고정된 하이퍼파라미터 추가

max_params.update(fixed_params)

In [43]:
# 최종 하이퍼파라미터 출력
max_params

{'bagging_fraction': 0.6213108174593661,
 'feature_fraction': 0.608712929970154,
 'lambda_l1': 0.7040436794880651,
 'lambda_l2': 0.9832619845547939,
 'min_child_samples': 9,
 'min_child_weight': 36.10036444740457,
 'num_leaves': 40,
 'objective': 'binary',
 'learning_rate': 0.005,
 'bagging_freq': 1,
 'force_row_wise': True,
 'random_state': 1991}

## 모델 훈련 및 성능 검증

In [48]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits=5 , shuffle = True , random_state= 1991)

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열

oof_val_preds = np.zeros(X.shape[0])

# OOF 방식으로 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0])

# OOF 방식으로 모델 훈련 ,검증 , 예측

for idx, (train , valid_idx) in enumerate(folds.split(X,y)):
    # 각 폴드를 구분하는 문구 출력
    print('#'*40 , f'폴드 {idx+1} / 폴드 {folds.n_splits}' , '#'*40)

    X_train , y_train = X[train_idx] , y[train_idx] # 훈련용 데이터
    X_valid , y_valid = X[valid_idx] , y[valid_idx] # 검증용 데이터

    # LightGBM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train , y_train) # LightGBM 전용 훈련 데이터셋
    dvalid = lgb.Dataset(X_valid , y_valid) # LightGBM 전용 검증 데이터셋

    # LightGBM 모델 훈련
    lgb_model = lgb.train(params = max_params , # 최적 하이퍼파라미터
                          train_set = dtrain, # 훈련 데이터 셋
                          num_boost_round= 2500, # 부스팅 반복 횟수
                          valid_sets= dvalid , # 성능 평가용 검증 데이터셋
                          feval = gini, # 검증용 평가지표
                          early_stopping_rounds= 300, # 조기종료 조건
                          verbose_eval = 100) # 100 번째 마다 점수 출력

    # 테스트 데이터를 활용해 OOF 예측
    oof_test_preds += lgb_model.predict(X_test) / folds.n_splits

    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측

    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    oof_test_preds_lgb = oof_test_preds
    # 검증 데이터 예측 확률에 대한 정규화 지니계수
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1}  지니계수 : {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\andyp\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17355, number of negative: 458815
[LightGBM] [Info] Total Bins 1558
[LightGBM] [Info] Number of data points in the train set: 476170, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274766
[LightGBM] [Info] Start training from score -3.274766
Training until validation scores don't improve for 300 rounds
[100]	valid_0's binary_logloss: 0.153929	valid_0's gini: 0.297826
[200]	valid_0's binary_logloss: 0.152575	valid_0's gini: 0.307987
[300]	valid_0's binary_logloss: 0.151732	valid_0's gini: 0.315646
[400]	valid_0's binary_logloss: 0.15115	valid_0's gini: 0.322832
[500]	valid_0's binary_logloss: 0.150688	valid_0's gini: 0.330498
[600]	valid_0's binary_logloss: 0.150302	valid_0's gini: 0.337661
[700]	valid_0's binary_logloss: 0.149967	valid_0's gini: 0.344703
[800]	valid_0's binary_logloss: 0.149665	valid_0's gini: 0.351174
[900]	valid_0's binary_logloss: 0.149386	valid_0's gini: 0.357542
[1000]	val

In [49]:
y.shape

(595212,)

In [50]:
oof_val_preds.shape

(595212,)

In [51]:
print('OOF 검증 데이터 지니계수 :' , eval_gini(y, oof_val_preds))

OOF 검증 데이터 지니계수 : 0.3949215102693418


## XGBoost 피처 엔지니어링

In [52]:
# LightGBM 용 gini() 함수
def gini(preds , dtrain):
    labels = dtrain.get_label()
    return 'gini' , eval_gini(labels , preds) , True

In [53]:
# XGBoost용 gini() 함수

def gini(preds , dtrain):
    labels = dtrain.get_label()
    return 'gini' , eval_gini(labels,preds)

## 하이퍼파라미터 최적화

### 데이터 셋 준비

In [55]:
import xgboost as xgb

from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터 , 검증 데이터 분리( 베이지안 최적화 수행용)

X_train , X_valid , y_train , y_valid = train_test_split(X,y, test_size=0.2 , random_state=0)

# 베이지안 최적화용 데이터셋

bayes_dtrain = xgb.DMatrix(X_train , y_train)
bayes_dvalid = xgb.DMatrix(X_valid, y_valid)

### 하이퍼파라미터 범위 설정

In [64]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
param_bounds = {'max_depth' : (4 , 8) ,
                'subsample' : (0.6 , 0.9),
                'colsample_bytree' : (0.7 , 1.0),
                'min_child_weight' : (5 , 7),
                'gamma' : (8 , 11),
                'reg_alpha' : (7 , 9) ,
                'reg_lambda' : (1.1 , 1.5),
                'scale_pos_weight' : (1.4 , 1.6)}


# 값이 고정된 하이퍼파라미터

fixed_params = {'objective' : 'binary:logistic' ,
                'learning_rate' : 0.02,
                'random_state' : 1991}

In [65]:
def eval_function(max_depth , subsample , colsample_bytree , min_child_weight , reg_alpha , gamma , reg_lambda , scale_pos_weight) :

    # 최적화하려는 평가지표(지니계수) 계산 함수

    # 베이지안 최적화를 수행할 하이퍼파라미터

    params = {'max_depth' : int(round(max_depth)) ,
              'subsample' : subsample,
              'colsample_bytree' : colsample_bytree ,
              'min_child_weight' : min_child_weight,
              'gamma' : gamma,
              'reg_alpha' : reg_alpha,
              'reg_lambda' : reg_lambda,
              'scale_pos_weight' : scale_pos_weight}

    # 값이 고정된 하이퍼파라미터도 추가
    params.update(fixed_params)

    print('하이퍼파라미터 : ' , params)

    # XGBoost 모델 훈련
    xgb_model = xgb.train(params = params ,
                          dtrain = bayes_dtrain,
                          num_boost_round= 2000,
                          evals = [(bayes_dvalid , ' bayes_dvalid')],
                          maximize = True,
                          feval = gini,
                          early_stopping_rounds= 200,
                          verbose_eval= False)

    best_iter = xgb_model.best_iteration # 최적 반복횟수
    # 검증 데이터로 예측 수행
    preds = xgb_model.predict(bayes_dvalid , iteration_range=(0, best_iter))

    # 지니계수 계산
    gini_score = eval_gini(y_valid, preds)
    print(f'지니계수 : {gini_score}\n')

    return gini_score

## 최적화 수행

In [ ]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성

optimizer = BayesianOptimization(f= eval_function, pbounds = param_bounds , random_state= 0)


# 베이지안 최적화 수행

optimizer.maximize(init_points= 3 , n_iter= 6)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------
하이퍼파라미터 :  {'max_depth': 6, 'subsample': 0.867531900234624, 'colsample_bytree': 0.8646440511781974, 'min_child_weight': 6.0897663659937935, 'gamma': 10.14556809911726, 'reg_alpha': 7.84730959867781, 'reg_lambda': 1.3583576452266626, 'scale_pos_weight': 1.4875174422525386, 'objective': 'binary:logistic', 'learning_rate': 0.02, 'random_state': 1991}


## 결과확인

In [ ]:
# 평가함수 점수가 최대일 때 하이퍼파라미터
max_params = optimizer.max['params']
max_params

In [ ]:
# 정수형 하이퍼파라미터 변환

max_params['max_depth'] = int(round(max_params['max_depth']))

# 값이 고정된 하이퍼파라미터 추가

max_params.update(fixed_params)
max_params

## 모델 훈련 및 성능 검증

In [ ]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기 생성
folds = StratifiedKFold(n_splits= 5 , shuffle= True , random_state= 1991)

# OOF 방식으로 훈련된 모델로 검증 데이터 타깃값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0])

# OOF 방식으로 훈련된 모델 훈련 , 검증 , 예측

for idx , (train_idx , valid_idx) in enumerate(folds.split(X,y)):
    # 각 폴드를 구분하는 문구 출력

    print('#' *40,  f'폴드 {idx+1} / 폴드 {folds.n_splits}' , '#'*40)


    # 훈련용 데이터, 검증용 데이터 설정
    X_train , y_train = X[train_idx] , y[train_idx]
    X_valid , y_valid = X[valid_idx] , y[valid_idx]

    #XGBoost 전용 데이터셋 생성

    dtrain = xgb.DMatrix(X_train , y_train)
    dvalid = xgb.DMatrix(X_valid , y_valid)
    dtest = xgb.DMatrix(X_test)

    #XGBoost 모델 훈련
    xgb_model = xgb.train(params = max_params,
                          dtrain = dtrain,
                          num_boost_round = 2000,
                          evals = [(dvalid , 'valid')],
                          maximize = True,
                          feval = gini,
                          early_stopping_rounds = 200,
                          verbose_eval = 100)

    # 모델 성능이 가장 좋을 때의 부스팅 반복 횟수 저장
    best_iter= xgb_model.best_iteration
    # 테스트 데이터를 활용해 OOF 예측

    oof_test_preds += xgb_model.predict(dtest, iteration_range = (0 , best_iter))/ folds.n_splits

    oof_test_preds_xgb = oof_test_preds
    # 모델 성능 평가를 위한 검증 데이터 타깃값 예측
    oof_val_preds[valid_idx] += xgb_model.predict(dvalid , iteration_range=(0, best_iter))


    # 검증 데이터 예측 확률에 대한 정규화 지니계수
    gini_score = eval_gini(y_valid , oof_val_preds[valid_idx])
    print(f'폴드 {idx+1}  지니계수 : {gini_score}\n')

In [ ]:
print('OOF 검증 데이터 지니계수 : ' , eval_gini(y , oof_val_preds))

In [ ]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')

## 성능 개선 3 : LightGBM 과 XGBoost 앙상블

In [ ]:
oof_test_preds = oof_test_preds_lgb *0.5 + oof_test_preds_xgb * 0.5


submission['target'] = oof_test_preds
submission.to_csv('submission.csv')

In [ ]:
submission